In [1]:
# imports
import pandas as pd
import os, ipdb, re
import random, evaluate
import string
import numpy as np
from collections import defaultdict
from tqdm import tqdm
from datasets import DatasetDict, Dataset, load_dataset
import wandb
import ast
import re, os
import subprocess

from fuzzywuzzy import fuzz
import matplotlib.pyplot as plt

/nfs/home/kabenamualus/anaconda3/envs/tdm/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# # dataset_path = "../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_ALL_TEMPLATE/fold1"
# # dataset_path = f"../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_50_PERCENT/fold1"
# # dataset_path = f"../data/LLLM_AUGMENTED_SUMMARIZED_ZEROSHOT_TDMS_50_PERCENT_LONG/fold1"
# # dataset_path = f"../data/LLLM_AUGMENTED_SUMMARIZED_ZEROSHOT_TDMS_50_PERCENT_DOCTEAT/fold1"
# dataset_path = f"../data/LLLM_AUGMENTED_SUMMARIZED_ZEROSHOT_TDMS_NO_TEMPLATE_DOCTEAT/fold1"

# dataset_dict = DatasetDict.load_from_disk(f"{dataset_path}")
    
# dfs = {split: dataset.to_pandas() for split, dataset in dataset_dict.items()}

In [3]:
def calculate_fuzz_ratio(text1, text2):
    return fuzz.ratio(str(text1).strip().lower(), str(text2).strip().lower())

def verify_data_integrity(dfs):

    zeroshot_paper_in_train = 0
    for paper in dfs['zeroshot'].id.unique():
        if paper in dfs['train'].id.to_list():
            zeroshot_paper_in_train += 1
        
    zeroshot_paper_in_validation = 0
    for paper in dfs['zeroshot'].id.unique():
        if paper in dfs['validation'].id.to_list():
            zeroshot_paper_in_validation += 1 

    train_paper_in_validation = 0
    for paper in dfs['train'].id.unique():
        if paper in dfs['validation'].id.to_list():
            train_paper_in_validation += 1   
        
    validation_paper_in_zeroshot = 0
    for paper in dfs['validation'].id.unique():
        if paper in dfs['zeroshot'].id.to_list():
            validation_paper_in_zeroshot += 1     
            
    print(f"zeroshot_paper_in_train: {zeroshot_paper_in_train}")
    print(f"zeroshot_paper_in_validation: {zeroshot_paper_in_validation}")
    print(f"train_paper_in_validation: {train_paper_in_validation}")
    print(f"validation_paper_in_zeroshot: {validation_paper_in_zeroshot}")
    
def remove_data_leakage(dfs, source, target, THRESHOLD=90):
    papers_to_drop = []
    missed_parsing_zero = 0
    
    dfs[source].drop_duplicates(subset='id', inplace=True)
    records = dfs[source].to_dict("records")

    for row in tqdm(records, total=len(records)):
        to_drop_paper_found = False 
        source_paper_id = row["id"]
        tdms = row["answer"]
        
        if tdms in "unanswerable" or tdms == "missed":
            # ipdb.set_trace()
            # if source_paper_id not in papers_to_drop:
            #     papers_to_drop.append(source_paper_id)
            continue 
        
        tdms = ast.literal_eval(row["answer"])
        
        for leaderboard in tdms:
            if to_drop_paper_found:
                break
            source_task = leaderboard["LEADERBOARD"]["Task"]
            source_dataset = leaderboard["LEADERBOARD"]["Dataset"]
            source_metric = leaderboard["LEADERBOARD"]["Metric"]
            source_score = leaderboard["LEADERBOARD"]["Score"]
 
            dfs[target].drop_duplicates(subset='id', inplace=True)
            target_records = dfs[target].to_dict("records")

            for row_target in target_records:
                
                target_paper_id = row_target["id"]
                target_tdms = row["answer"]
                
                if target_tdms == "unanswerable" or target_tdms == "missed":
                    # ipdb.set_trace()
                    continue 
                
                target_tdms = ast.literal_eval(row["answer"])
                
                for leaderboard in target_tdms:
                    target_task = leaderboard["LEADERBOARD"]["Task"]
                    target_dataset = leaderboard["LEADERBOARD"]["Dataset"]
                    target_metric = leaderboard["LEADERBOARD"]["Metric"]
                    target_score = leaderboard["LEADERBOARD"]["Score"]
                    
                    # A) Exact 
                    if (source_task == target_task and source_dataset == target_dataset and source_metric == target_metric) or (target_paper_id == source_paper_id):
                        if source_paper_id not in papers_to_drop:
                            papers_to_drop.append(source_paper_id)
                        to_drop_paper_found = True 
                        break
                
                    if to_drop_paper_found:
                        break
            
    
                    # # B) Partial  or # A) Exact 
                    # if (calculate_fuzz_ratio(train_task, zeroshot_task) >= THRESHOLD and calculate_fuzz_ratio(train_dataset, zeroshot_dataset) >= THRESHOLD and calculate_fuzz_ratio(train_metric, zeroshot_metric) >= THRESHOLD) or (train_task == zeroshot_task and train_dataset == zeroshot_dataset and train_metric == zeroshot_metric):
                    #     if paper_id not in papers_to_drop:
                    #         papers_to_drop.append(paper_id)
                    #         # ipdb.set_trace()
                    #         break 
                    
    print(f"Missed_parsing_zero {missed_parsing_zero}") 
    print(f"Lenght papers_to_drop in {source} based on {target}: {len(papers_to_drop)}")        
    
    return papers_to_drop

# No Templates

## DOCTEAT

### STRICT

In [4]:
dataset_path = f"../data/LLLM_AUGMENTED_SUMMARIZED_ZEROSHOT_TDMS_NO_TEMPLATE_DOCTEAT/fold1"

dataset_dict = DatasetDict.load_from_disk(f"{dataset_path}")
    
dfs = {split: dataset.to_pandas() for split, dataset in dataset_dict.items()}

In [5]:
dfs['train'].describe()

,prompt lenght,__index_level_0__
count,150870.000000,150870.000000
mean,485.707298,75434.500000
std,398.720864,43552.561894
min,27.000000,0.000000
25%,216.000000,37717.250000
50%,392.000000,75434.500000
75%,640.000000,113151.750000
max,7344.000000,150869.000000


In [6]:
dfs['validation'].describe()

,prompt lenght,__index_level_0__
count,20205.000000,20205.00000
mean,473.135857,10102.00000
std,352.229574,5832.82543
min,26.000000,0.00000
25%,222.000000,5051.00000
50%,387.000000,10102.00000
75%,614.000000,15153.00000
max,3431.000000,20204.00000


In [7]:
dfs['zeroshot'].describe()

,prompt lenght,__index_level_0__
count,13890.000000,13890.000000
mean,444.615551,6944.500000
std,405.101173,4009.841954
min,29.000000,0.000000
25%,199.000000,3472.250000
50%,342.000000,6944.500000
75%,582.000000,10416.750000
max,7344.000000,13889.000000


In [8]:
zeroshot_papers_to_drop_no_lb = remove_data_leakage(dfs, source="zeroshot", target="train", THRESHOLD=90)

100%|██████████| 926/926 [09:11<00:00,  1.68it/s] 

Missed_parsing_zero 0
Lenght papers_to_drop in zeroshot based on train: 573


In [9]:
# validation_papers_to_drop_no_lb = remove_data_leakage(dfs, source="validation", target="zeroshot", THRESHOLD=90)

dfs['zeroshot'].drop_duplicates(subset='id', inplace=True)
records = dfs['zeroshot'].to_dict("records")
validation_papers_to_drop_no_lb = set()
for row in tqdm(records, total=len(records)):
    # if row['id'] in dfs['train']['id'].to_list():
    #     duplicate+=1
    if row['id'] in dfs['validation']['id'].to_list():
        validation_papers_to_drop_no_lb.add(row['id'])
        
print(len(validation_papers_to_drop_no_lb))

100%|██████████| 926/926 [00:00<00:00, 6354.35it/s]

408


In [10]:
# Filtering the DataFrame
dfs['zeroshot'] = dfs['zeroshot'][~dfs['zeroshot']['id'].isin(zeroshot_papers_to_drop_no_lb)]
dfs['zeroshot'].drop_duplicates(subset='id', inplace=True)
dfs['zeroshot'].describe()

,prompt lenght,__index_level_0__
count,353.000000,353.00000
mean,197.560907,11235.00000
std,99.668184,1530.69837
min,29.000000,8595.00000
25%,138.000000,9915.00000
50%,192.000000,11235.00000
75%,234.000000,12555.00000
max,677.000000,13875.00000


In [11]:
# Filtering the DataFrame
dfs['validation'] = dfs['validation'][~dfs['validation']['id'].isin(validation_papers_to_drop_no_lb)]
dfs['validation'].drop_duplicates(subset='id', inplace=True)
dfs['validation'].describe()

,prompt lenght,__index_level_0__
count,939.000000,939.000000
mean,568.654952,7434.408946
std,358.102912,4300.934395
min,26.000000,0.000000
25%,325.000000,3712.500000
50%,497.000000,7425.000000
75%,707.500000,11137.500000
max,3431.000000,14895.000000


In [12]:
# Filtering the DataFrame
dfs['train'].drop_duplicates(subset='id', inplace=True)
dfs['train'].describe()

,prompt lenght,__index_level_0__
count,10058.000000,10058.000000
mean,485.707298,75427.500000
std,398.739365,43554.582566
min,27.000000,0.000000
25%,216.000000,37713.750000
50%,392.000000,75427.500000
75%,640.000000,113141.250000
max,7344.000000,150855.000000


In [13]:
dfs['train'].drop(["template", "__index_level_0__"], inplace=True, axis=1)
dfs['validation'].drop(["template", "__index_level_0__"], inplace=True, axis=1)
dfs['zeroshot'].drop(["template", "__index_level_0__"], inplace=True, axis=1)

In [14]:
dfs['train'].tail()

,id,prompt,answer,prompt lenght
150795,1401.6576.pdf,Adding modular predicates to first-order fragm...,unanswerable,206
150810,1304.4691.pdf,Efficient Calculation of Determinants of Symbo...,unanswerable,128
150825,1001.3388.pdf,Approximate Privacy: PARs for Set Problems In ...,unanswerable,183
150840,0706.2010.pdf,Information-theoretic securitywithout an hones...,unanswerable,131
150855,1111.2942.pdf,Down the Rabbit Hole: Robust Proximity Search ...,unanswerable,232


In [15]:
dfs['train'].id.nunique()

10058

In [16]:
def verify_data_integrity(dfs):

    zeroshot_paper_in_train = 0
    for paper in dfs['zeroshot'].id.unique():
        if paper in dfs['train'].id.to_list():
            zeroshot_paper_in_train += 1
        
    zeroshot_paper_in_validation = 0
    for paper in dfs['zeroshot'].id.unique():
        if paper in dfs['validation'].id.to_list():
            zeroshot_paper_in_validation += 1 

    train_paper_in_validation = 0
    for paper in dfs['train'].id.unique():
        if paper in dfs['validation'].id.to_list():
            train_paper_in_validation += 1   
        
    validation_paper_in_zeroshot = 0
    for paper in dfs['validation'].id.unique():
        if paper in dfs['zeroshot'].id.to_list():
            validation_paper_in_zeroshot += 1     
            
    print(f"zeroshot_paper_in_train: {zeroshot_paper_in_train}")
    print(f"zeroshot_paper_in_validation: {zeroshot_paper_in_validation}")
    print(f"train_paper_in_validation: {train_paper_in_validation}")
    print(f"validation_paper_in_zeroshot: {validation_paper_in_zeroshot}")

In [17]:
verify_data_integrity(dfs)

zeroshot_paper_in_train: 0
zeroshot_paper_in_validation: 0
train_paper_in_validation: 0
validation_paper_in_zeroshot: 0


In [18]:
dataset = DatasetDict({
        "train": Dataset.from_pandas(dfs['train']),
        "validation": Dataset.from_pandas(dfs['validation']),
        "zeroshot": Dataset.from_pandas(dfs['zeroshot'])
    })

dataset.save_to_disk("../data/LEADERBOARDS_DOCTEAT_TDMS_NO_TEMPLATES_STRICT")

Saving the dataset (0/1 shards):   0%|          | 0/10058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/939 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/353 [00:00<?, ? examples/s]

### Template Specific

In [19]:
for squad_id in range(1, 9):
    print(f"SQUAD: {squad_id}")
    dataset_path = f"../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_SQUAD_{squad_id}_DOCTEAT/fold1"

    dataset_dict = DatasetDict.load_from_disk(f"{dataset_path}")
        
    dfs = {split: dataset.to_pandas() for split, dataset in dataset_dict.items()}
    
    zeroshot_papers_to_drop_no_lb = remove_data_leakage(dfs, source="zeroshot", target="train", THRESHOLD=90)
    
    dfs['zeroshot'].drop_duplicates(subset='id', inplace=True)
    records = dfs['zeroshot'].to_dict("records")
    validation_papers_to_drop_no_lb = set()
    for row in tqdm(records, total=len(records)):
        # if row['id'] in dfs['train']['id'].to_list():
        #     duplicate+=1
        if row['id'] in dfs['validation']['id'].to_list():
            validation_papers_to_drop_no_lb.add(row['id'])
            
    print(len(validation_papers_to_drop_no_lb))
    
    # Filtering the DataFrame
    dfs['train'].drop_duplicates(subset='id', inplace=True)
    display(dfs['train'].describe())

    # Filtering the DataFrame
    dfs['validation'] = dfs['validation'][~dfs['validation']['id'].isin(validation_papers_to_drop_no_lb)]
    dfs['validation'].drop_duplicates(subset='id', inplace=True)
    display(dfs['validation'].describe())

    # Filtering the DataFrame
    dfs['zeroshot'] = dfs['zeroshot'][~dfs['zeroshot']['id'].isin(zeroshot_papers_to_drop_no_lb)]
    dfs['zeroshot'].drop_duplicates(subset='id', inplace=True)
    display(dfs['zeroshot'].describe())
    
    dfs['train'].drop(["template", "__index_level_0__"], inplace=True, axis=1)
    dfs['validation'].drop(["template", "__index_level_0__"], inplace=True, axis=1)
    dfs['zeroshot'].drop(["template", "__index_level_0__"], inplace=True, axis=1)

    verify_data_integrity(dfs)
    
    dataset = DatasetDict({
        "train": Dataset.from_pandas(dfs['train']),
        "validation": Dataset.from_pandas(dfs['validation']),
        "zeroshot": Dataset.from_pandas(dfs['zeroshot'])
    })

    dataset.save_to_disk(f"../data/LEADERBOARDS_DOCTEAT_TDMS_NO_TEMPLATES_SQUAD_{squad_id}_STRICT")

SQUAD: 1


  0%|          | 0/926 [00:00<?, ?it/s]

 28%|██▊       | 262/926 [05:04<10:02,  1.10it/s] 

In [ ]:
for drop_id in range(1, 8):
    print(f"DROP: {drop_id}")
    dataset_path = f"../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_DROP_{drop_id}_DOCTEAT/fold1"

    dataset_dict = DatasetDict.load_from_disk(f"{dataset_path}")
        
    dfs = {split: dataset.to_pandas() for split, dataset in dataset_dict.items()}
    
    zeroshot_papers_to_drop_no_lb = remove_data_leakage(dfs, source="zeroshot", target="train", THRESHOLD=90)
    
    dfs['zeroshot'].drop_duplicates(subset='id', inplace=True)
    records = dfs['zeroshot'].to_dict("records")
    validation_papers_to_drop_no_lb = set()
    for row in tqdm(records, total=len(records)):
        # if row['id'] in dfs['train']['id'].to_list():
        #     duplicate+=1
        if row['id'] in dfs['validation']['id'].to_list():
            validation_papers_to_drop_no_lb.add(row['id'])
            
    print(len(validation_papers_to_drop_no_lb))
    
    # Filtering the DataFrame
    dfs['train'].drop_duplicates(subset='id', inplace=True)
    display(dfs['train'].describe())

    # Filtering the DataFrame
    dfs['validation'] = dfs['validation'][~dfs['validation']['id'].isin(validation_papers_to_drop_no_lb)]
    dfs['validation'].drop_duplicates(subset='id', inplace=True)
    display(dfs['validation'].describe())

    # Filtering the DataFrame
    dfs['zeroshot'] = dfs['zeroshot'][~dfs['zeroshot']['id'].isin(zeroshot_papers_to_drop_no_lb)]
    dfs['zeroshot'].drop_duplicates(subset='id', inplace=True)
    display(dfs['zeroshot'].describe())
    
    dfs['train'].drop(["template", "__index_level_0__"], inplace=True, axis=1)
    dfs['validation'].drop(["template", "__index_level_0__"], inplace=True, axis=1)
    dfs['zeroshot'].drop(["template", "__index_level_0__"], inplace=True, axis=1)

    verify_data_integrity(dfs)
    
    dataset = DatasetDict({
        "train": Dataset.from_pandas(dfs['train']),
        "validation": Dataset.from_pandas(dfs['validation']),
        "zeroshot": Dataset.from_pandas(dfs['zeroshot'])
    })

    dataset.save_to_disk(f"../data/LEADERBOARDS_DOCTEAT_TDMS_NO_TEMPLATES_DROP_{drop_id}_STRICT")

100%|██████████| 926/926 [09:08<00:00,  1.69it/s] 


Missed_parsing_zero 0
Lenght papers_to_drop in zeroshot based on train: 573


100%|██████████| 926/926 [00:00<00:00, 75687.92it/s]

408


,prompt lenght,__index_level_0__
count,10058.000000,10058.000000
mean,488.707298,75435.500000
std,398.739365,43554.582566
min,30.000000,8.000000
25%,219.000000,37721.750000
50%,395.000000,75435.500000
75%,643.000000,113149.250000
max,7347.000000,150863.000000


...

,prompt lenght,__index_level_0__
count,353.000000,353.000000
mean,199.560907,11248.988669
std,99.668184,1530.687806
min,31.000000,8609.000000
25%,140.000000,9929.000000
50%,194.000000,11249.000000
75%,236.000000,12569.000000
max,679.000000,13889.000000


zeroshot_paper_in_train: 0
zeroshot_paper_in_validation: 0
train_paper_in_validation: 0
validation_paper_in_zeroshot: 0


Saving the dataset (0/1 shards):   0%|          | 0/10058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/939 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/353 [00:00<?, ? examples/s]

## DOCTEAT

### RELAX

In [ ]:
dataset_path = f"../data/LLLM_AUGMENTED_SUMMARIZED_ZEROSHOT_TDMS_NO_TEMPLATE_DOCTEAT/fold1"

dataset_dict = DatasetDict.load_from_disk(f"{dataset_path}")
    
dfs = {split: dataset.to_pandas() for split, dataset in dataset_dict.items()}

In [ ]:
dfs['train'].describe()

,prompt lenght,__index_level_0__
count,150870.000000,150870.000000
mean,485.707298,75434.500000
std,398.720864,43552.561894
min,27.000000,0.000000
25%,216.000000,37717.250000
50%,392.000000,75434.500000
75%,640.000000,113151.750000
max,7344.000000,150869.000000


In [ ]:
dfs['validation'].describe()

,prompt lenght,__index_level_0__
count,20205.000000,20205.00000
mean,473.135857,10102.00000
std,352.229574,5832.82543
min,26.000000,0.00000
25%,222.000000,5051.00000
50%,387.000000,10102.00000
75%,614.000000,15153.00000
max,3431.000000,20204.00000


In [ ]:
dfs['zeroshot'].describe()

,prompt lenght,__index_level_0__
count,13890.000000,13890.000000
mean,444.615551,6944.500000
std,405.101173,4009.841954
min,29.000000,0.000000
25%,199.000000,3472.250000
50%,342.000000,6944.500000
75%,582.000000,10416.750000
max,7344.000000,13889.000000


In [ ]:
zeroshot_papers_to_drop_no_lb = remove_data_leakage(dfs, source="zeroshot", target="train", THRESHOLD=90)

 35%|███▌      | 328/926 [06:04<04:14,  2.35it/s] 

In [ ]:
# validation_papers_to_drop_no_lb = remove_data_leakage(dfs, source="validation", target="zeroshot", THRESHOLD=90)

records = dfs['zeroshot'].to_dict("records")
validation_papers_to_drop_no_lb = set()
for row in tqdm(records, total=len(records)):
    # if row['id'] in dfs['train']['id'].to_list():
    #     duplicate+=1
    if row['id'] in dfs['validation']['id'].to_list():
        validation_papers_to_drop_no_lb.add(row['id'])
        
print(len(validation_papers_to_drop_no_lb))

100%|██████████| 926/926 [00:00<00:00, 6443.53it/s]

408


In [ ]:
# Filtering the DataFrame
dfs['zeroshot'] = dfs['zeroshot'][~dfs['zeroshot']['id'].isin(zeroshot_papers_to_drop_no_lb)]
dfs['zeroshot'].describe()

,prompt lenght,__index_level_0__
count,353.000000,353.00000
mean,197.560907,11235.00000
std,99.668184,1530.69837
min,29.000000,8595.00000
25%,138.000000,9915.00000
50%,192.000000,11235.00000
75%,234.000000,12555.00000
max,677.000000,13875.00000


In [ ]:
# Filtering the DataFrame
dfs['validation'] = dfs['validation'][~dfs['validation']['id'].isin(validation_papers_to_drop_no_lb)]
dfs['validation'].describe()

,prompt lenght,__index_level_0__
count,939.000000,939.000000
mean,568.654952,7434.408946
std,358.102912,4300.934395
min,26.000000,0.000000
25%,325.000000,3712.500000
50%,497.000000,7425.000000
75%,707.500000,11137.500000
max,3431.000000,14895.000000


In [ ]:
# Filtering the DataFrame
dfs['train'].describe()

,prompt lenght,__index_level_0__
count,10058.000000,10058.000000
mean,485.707298,75427.500000
std,398.739365,43554.582566
min,27.000000,0.000000
25%,216.000000,37713.750000
50%,392.000000,75427.500000
75%,640.000000,113141.250000
max,7344.000000,150855.000000


In [ ]:
dfs['train'].drop(["template", "__index_level_0__"], inplace=True, axis=1)
dfs['validation'].drop(["template", "__index_level_0__"], inplace=True, axis=1)
dfs['zeroshot'].drop(["template", "__index_level_0__"], inplace=True, axis=1)

In [ ]:
dfs['train'].tail()

,id,prompt,answer,prompt lenght
150795,1401.6576.pdf,Adding modular predicates to first-order fragm...,unanswerable,206
150810,1304.4691.pdf,Efficient Calculation of Determinants of Symbo...,unanswerable,128
150825,1001.3388.pdf,Approximate Privacy: PARs for Set Problems In ...,unanswerable,183
150840,0706.2010.pdf,Information-theoretic securitywithout an hones...,unanswerable,131
150855,1111.2942.pdf,Down the Rabbit Hole: Robust Proximity Search ...,unanswerable,232


In [ ]:
dfs['train'].id.nunique()

10058

In [ ]:
verify_data_integrity(dfs)

zeroshot_paper_in_train: 0
zeroshot_paper_in_validation: 0
train_paper_in_validation: 0
validation_paper_in_zeroshot: 0


In [ ]:
dataset = DatasetDict({
        "train": Dataset.from_pandas(dfs['train']),
        "validation": Dataset.from_pandas(dfs['validation']),
        "zeroshot": Dataset.from_pandas(dfs['zeroshot'])
    })

dataset.save_to_disk("../data/LEADERBOARDS_DOCTEAT_TDMS_NO_TEMPLATES_RELAX")

Saving the dataset (0/1 shards):   0%|          | 0/10058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/939 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/353 [00:00<?, ? examples/s]

### Templates Specific

In [ ]:
for squad_id in range(1, 9):
    print(f"SQUAD: {squad_id}")
    dataset_path = f"../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_SQUAD_{squad_id}_DOCTEAT/fold1"

    dataset_dict = DatasetDict.load_from_disk(f"{dataset_path}")
        
    dfs = {split: dataset.to_pandas() for split, dataset in dataset_dict.items()}
    
    zeroshot_papers_to_drop_no_lb = remove_data_leakage(dfs, source="zeroshot", target="train", THRESHOLD=90)
    
    records = dfs['zeroshot'].to_dict("records")
    validation_papers_to_drop_no_lb = set()
    for row in tqdm(records, total=len(records)):
        # if row['id'] in dfs['train']['id'].to_list():
        #     duplicate+=1
        if row['id'] in dfs['validation']['id'].to_list():
            validation_papers_to_drop_no_lb.add(row['id'])
            
    print(len(validation_papers_to_drop_no_lb))
    
    # Filtering the DataFrame
    display(dfs['train'].describe())

    # Filtering the DataFrame
    dfs['validation'] = dfs['validation'][~dfs['validation']['id'].isin(validation_papers_to_drop_no_lb)]
    display(dfs['validation'].describe())

    # Filtering the DataFrame
    dfs['zeroshot'] = dfs['zeroshot'][~dfs['zeroshot']['id'].isin(zeroshot_papers_to_drop_no_lb)]
    display(dfs['zeroshot'].describe())
    
    dfs['train'].drop(["template", "__index_level_0__"], inplace=True, axis=1)
    dfs['validation'].drop(["template", "__index_level_0__"], inplace=True, axis=1)
    dfs['zeroshot'].drop(["template", "__index_level_0__"], inplace=True, axis=1)

    verify_data_integrity(dfs)
    
    dataset = DatasetDict({
        "train": Dataset.from_pandas(dfs['train']),
        "validation": Dataset.from_pandas(dfs['validation']),
        "zeroshot": Dataset.from_pandas(dfs['zeroshot'])
    })

    dataset.save_to_disk(f"../data/LEADERBOARDS_DOCTEAT_TDMS_NO_TEMPLATES_SQUAD_{squad_id}_RELAX")

100%|██████████| 926/926 [09:07<00:00,  1.69it/s] 


In [ ]:
for drop_id in range(1, 8):
    print(f"DROP: {drop_id}")
    dataset_path = f"../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_DROP_{drop_id}_DOCTEAT/fold1"

    dataset_dict = DatasetDict.load_from_disk(f"{dataset_path}")
        
    dfs = {split: dataset.to_pandas() for split, dataset in dataset_dict.items()}
    
    zeroshot_papers_to_drop_no_lb = remove_data_leakage(dfs, source="zeroshot", target="train", THRESHOLD=90)
    
    dfs['zeroshot'].drop_duplicates(subset='id', inplace=True)
    records = dfs['zeroshot'].to_dict("records")
    validation_papers_to_drop_no_lb = set()
    for row in tqdm(records, total=len(records)):
        # if row['id'] in dfs['train']['id'].to_list():
        #     duplicate+=1
        if row['id'] in dfs['validation']['id'].to_list():
            validation_papers_to_drop_no_lb.add(row['id'])
            
    print(len(validation_papers_to_drop_no_lb))
    
    # Filtering the DataFrame
    display(dfs['train'].describe())

    # Filtering the DataFrame
    dfs['validation'] = dfs['validation'][~dfs['validation']['id'].isin(validation_papers_to_drop_no_lb)]
    display(dfs['validation'].describe())

    # Filtering the DataFrame
    dfs['zeroshot'] = dfs['zeroshot'][~dfs['zeroshot']['id'].isin(zeroshot_papers_to_drop_no_lb)]
    display(dfs['zeroshot'].describe())
    
    dfs['train'].drop(["template", "__index_level_0__"], inplace=True, axis=1)
    dfs['validation'].drop(["template", "__index_level_0__"], inplace=True, axis=1)
    dfs['zeroshot'].drop(["template", "__index_level_0__"], inplace=True, axis=1)

    verify_data_integrity(dfs)
    
    dataset = DatasetDict({
        "train": Dataset.from_pandas(dfs['train']),
        "validation": Dataset.from_pandas(dfs['validation']),
        "zeroshot": Dataset.from_pandas(dfs['zeroshot'])
    })

    dataset.save_to_disk(f"../data/LEADERBOARDS_DOCTEAT_TDMS_NO_TEMPLATES_DROP_{drop_id}_RELAX")